Steps to Improve the Model:
Data Preparation:

Use your manually mapped datasets to create a labeled dataset for training the model.
Feature Engineering:

Instead of using raw text similarity (TF-IDF and cosine similarity), create features that can better represent the relationship between source and target categories. This could include embedding-based features, custom text similarity features, etc.
Model Training:

Train a supervised learning model using the labeled data. Models such as Random Forests, Gradient Boosting Machines, or even deep learning models like LSTMs or Transformers could be useful, depending on the size and complexity of your data.
Model Evaluation:

Evaluate the trained model's performance using a validation set. Use metrics like accuracy, F1-score, or other relevant metrics to assess how well the model is performing.
Prediction and Mapping:

Use the trained model to predict the mappings for any new data.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

# Load the manually mapped datasets
design_within_reach_mapped = pd.read_csv('Design Within Reach.csv')
discount_school_supply_mapped = pd.read_csv('Discount_School_Supply.csv')

# Combine the datasets for training
combined_data = pd.concat([design_within_reach_mapped, discount_school_supply_mapped])

# Preprocessing function to clean text data
def preprocess_text(text):
    if isinstance(text, str):
        # Lowercase the text
        text = text.lower()
        # Remove special characters
        text = ''.join(e for e in text if e.isalnum() or e.isspace())
        return text
    else:
        return ''

# Apply preprocessing
combined_data['src_pt'] = combined_data['src_pt'].apply(preprocess_text)
combined_data['src_cat'] = combined_data['src_cat'].apply(preprocess_text)
combined_data['src_sc'] = combined_data['src_sc'].apply(preprocess_text)

# Combine the text features
combined_data['combined'] = combined_data['src_pt'] + ' ' + combined_data['src_cat'] + ' ' + combined_data['src_sc']

# Encode target labels (productType, category, subCategory)
le_pt = LabelEncoder()
le_cat = LabelEncoder()
le_sc = LabelEncoder()

combined_data['ent_pt_2_encoded'] = le_pt.fit_transform(combined_data['ent_pt_2'])
combined_data['ent_cat_2_encoded'] = le_cat.fit_transform(combined_data['ent_cat_2'])
combined_data['ent_sc_2_encoded'] = le_sc.fit_transform(combined_data['ent_sc_2'])


In [2]:
# Split the data into training and testing sets
X_train, X_test, y_train_pt, y_test_pt, y_train_cat, y_test_cat, y_train_sc, y_test_sc = train_test_split(
    combined_data['combined'], 
    combined_data['ent_pt_2_encoded'], 
    combined_data['ent_cat_2_encoded'], 
    combined_data['ent_sc_2_encoded'], 
    test_size=0.2, 
    random_state=42
)

# Create a pipeline for text processing and classification
pipeline_pt = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_cat = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_sc = Pipeline([
    ('tfidf', TfidfVectorizer()), 
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Train the models
pipeline_pt.fit(X_train, y_train_pt)
pipeline_cat.fit(X_train, y_train_cat)
pipeline_sc.fit(X_train, y_train_sc)


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier(random_state=42))])

In [3]:
# Predict and evaluate productType
y_pred_pt = pipeline_pt.predict(X_test)
print("Product Type Classification Report")
print(classification_report(y_test_pt, y_pred_pt, target_names=le_pt.classes_))
print("Accuracy:", accuracy_score(y_test_pt, y_pred_pt))

# Predict and evaluate category
y_pred_cat = pipeline_cat.predict(X_test)
print("Category Classification Report")
print(classification_report(y_test_cat, y_pred_cat, target_names=le_cat.classes_))
print("Accuracy:", accuracy_score(y_test_cat, y_pred_cat))

# Predict and evaluate subCategory
y_pred_sc = pipeline_sc.predict(X_test)
print("SubCategory Classification Report")
print(classification_report(y_test_sc, y_pred_sc, target_names=le_sc.classes_))
print("Accuracy:", accuracy_score(y_test_sc, y_pred_sc))


Product Type Classification Report


ValueError: Number of classes, 11, does not match size of target_names, 13. Try specifying the labels parameter